<a href="https://colab.research.google.com/github/anu90dl/NeuralNetwork/blob/main/MnistFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load the libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
input_size = 784        #Number of input neurons (image pixels)
hidden_size = 400       #Number of hidden neurons
out_size = 10           #Number of classes (0-9)
epochs = 10            #How many times we pass our entire dataset into our network
batch_size = 100        #Input size of the data during one iteration
learning_rate = 0.001

In [3]:
#load train and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 6088609.57it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 346267.20it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 834323.24it/s] 


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 9568321.83it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
#Define the neural network class
class Net(nn.Module):

  def __init__(self,input_size,hidden_size,out_size):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.fc2= nn.Linear(hidden_size,hidden_size)
    self.fc3 = nn.Linear(hidden_size,out_size)
    self.relu = nn.ReLU()
    self.init_weights()

  def init_weights(self):
    nn.init.kaiming_normal_(self.fc1.weight)
    nn.init.kaiming_normal_(self.fc2.weight)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    return out

In [9]:
#load the loss function and optimizer
net = Net(input_size,hidden_size,out_size)
CUDA = torch.cuda.is_available()
if CUDA:
  net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=learning_rate)

In [10]:
#train the model
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

for epoch in range(epochs):
  correct_train = 0
  running_loss = 0
  for i,(images,labels) in enumerate(train_loader):
    images = images.view(-1,28*28)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()

    outputs = net(images)
    _,predicted = torch.max(outputs.data,1)
    correct_train = (labels.cpu()==predicted.cpu()).sum()
    loss = criterion(predicted,labels)
    running_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"the loss for this epoch {epoch +1}/{epochs} is {running_loss} with accuracy {100*correct_train.double()/len(train_dataset)}")
  print("Done Training")

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [ ]:
#test the dataset
with torch.no_grad():
  correct_test = 0
  for i,(images,labels) in enumerate(test_loader):
    images = images.view(-1,28*28)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()
    outputs = net(images)
    _,test_predict = torch.max(outputs.data,1)
    correct_test += (labels.cpu() == test_predict.cpu()).sum().item()
  print(f"Accuracy of the network on 1000 test images {100*correct_test/len(test_dataset)}")